In [0]:
%matplotlib inline

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset
print(torch.cuda.is_available())

True


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


# Model

In [4]:
import torch.nn as nn

import torch
import numpy as np
import torch.nn.functional as F
from torch.nn.init import normal, constant

model = torchvision.models.resnet18(pretrained=True)
model.fc = nn.Linear(512, 256)
normal(model.fc.weight, 0.0, 0.01)
constant(model.fc.bias, 0.0)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 46827520/46827520 [00:01<00:00, 25242979.00it/s]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  # This is added back by InteractiveShellApp.init_path()


In [5]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

# Optimizer

In [0]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Image Loaders

In [0]:
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')


def accimage_loader(path):
    import accimage
    try:
        return accimage.Image(path)
    except IOError:
        # Potentially a decoding problem, fall back to PIL.Image
        return pil_loader(path)


def default_loader(path):
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)

# Custom dataset class

In [0]:
import os
import os.path
from torch.utils.data import Dataset, DataLoader
from PIL import Image

def makeDataset(dir):
    classes = [d for d in os.listdir(dir) if os.path.isdir(os.path.join(dir, d))]
    classes.sort()
    class_to_idx = {classes[i]: i for i in range(len(classes))}
    images = []
    for target in sorted(class_to_idx.keys()):
        d = os.path.join(dir, target)
        if not os.path.isdir(d):
            continue
        for root, _, fnames in sorted(os.walk(d)):
            for fname in sorted(fnames):
                path = os.path.join(root, fname)
                item = (path, class_to_idx[target])
                images.append(item)

    return images
  
class myDataset(Dataset):
  
  def __init__(self, root_dir, loader= default_loader, transform=None):
    super(myDataset, self).__init__()
    self.root_dir = root_dir
    self.transform = transform
    self.loader = loader
    samples = makeDataset(root_dir)
    self.samples = samples
    
  def __len__(self):
    return len(self.samples)
  
  def __getitem__(self, index):
    x,y = self.samples[index]
    sample = self.loader(x)
    if self.transform is not None:
       sample = self.transform(sample)

    return sample, y

# Dataset and Dataloader

In [0]:
#transform2 = transforms.Compose([transforms.RandomHorizontalFlip(p=1), transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
import numpy as np
from random import sample
import random
trans11 = transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(p=0),
        transforms.CenterCrop(size=224),
        transforms.ToTensor()
    ])
trans21 = transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(p=1),
        transforms.CenterCrop(size=224),
        transforms.ToTensor()
    ])

trainset1 = myDataset('/content/256_ObjectCategories', transform=trans11)
trainset2 = myDataset('/content/256_ObjectCategories', transform=trans21)
testset = myDataset('/content/Test/test', transform=trans11)


tset = ConcatDataset((trainset1, trainset2))

tloader = torch.utils.data.DataLoader(tset, batch_size=64,
                                          shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                          shuffle=True, num_workers=2)

# Training Model

In [19]:
for epoch in range(1):
  running_loss = 0.0
  i=0;
  for data in tloader:
    x,y=data[0].to(device), data[1].to(device)
    output=model(x)
    loss=criterion(output,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    running_loss += loss.item()
    if i % 10 == 9:    # print every 47th mini-batch (47*5 total minis)
      print('[%d, %5d] loss: %.3f' %
      (epoch + 1, i + 1, running_loss/10))
      running_loss = 0.0
            
    i=i+1

    
torch.save(model.state_dict(), '/content/gdrive/My Drive/caltech00.pth')
print('Finished Training')


[1,    10] loss: 1.035
[1,    20] loss: 1.046
[1,    30] loss: 0.935
[1,    40] loss: 0.941
[1,    50] loss: 0.956
[1,    60] loss: 0.981
[1,    70] loss: 0.879
[1,    80] loss: 1.071
[1,    90] loss: 0.960
[1,   100] loss: 0.940
[1,   110] loss: 1.057
[1,   120] loss: 0.920
[1,   130] loss: 0.975
[1,   140] loss: 0.887
[1,   150] loss: 0.876
[1,   160] loss: 0.845
[1,   170] loss: 1.019
[1,   180] loss: 0.844
[1,   190] loss: 0.947
[1,   200] loss: 0.872
[1,   210] loss: 0.951
[1,   220] loss: 0.916
[1,   230] loss: 0.888
[1,   240] loss: 0.830
[1,   250] loss: 0.873
[1,   260] loss: 0.852
[1,   270] loss: 0.866
[1,   280] loss: 0.914
[1,   290] loss: 0.881
[1,   300] loss: 0.881
[1,   310] loss: 0.941
[1,   320] loss: 0.888
[1,   330] loss: 0.820
[1,   340] loss: 0.862
[1,   350] loss: 0.891
[1,   360] loss: 0.763
[1,   370] loss: 0.825
[1,   380] loss: 0.945
[1,   390] loss: 0.871
[1,   400] loss: 0.768
[1,   410] loss: 0.867
[1,   420] loss: 0.876
[1,   430] loss: 0.878
[1,   440] 

In [20]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
for epoch in range(2):
  running_loss = 0.0
  i=0;
  for data in tloader:
    x,y=data[0].to(device), data[1].to(device)
    output=model(x)
    loss=criterion(output,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    running_loss += loss.item()
    if i % 46 == 45:    # print every 47th mini-batch (47*5 total minis)
      print('[%d, %5d] loss: %.3f' %
      (epoch + 1, i + 1, running_loss/46))
      running_loss = 0.0
            
    i=i+1

    
torch.save(model.state_dict(), '/content/gdrive/My Drive/caltech001.pth')
print('Finished Training')


[1,    46] loss: 0.477
[1,    92] loss: 0.402
[1,   138] loss: 0.375
[1,   184] loss: 0.373
[1,   230] loss: 0.354
[1,   276] loss: 0.343
[1,   322] loss: 0.330
[1,   368] loss: 0.317
[1,   414] loss: 0.310
[1,   460] loss: 0.310
[1,   506] loss: 0.307
[1,   552] loss: 0.310
[1,   598] loss: 0.309
[1,   644] loss: 0.285
[1,   690] loss: 0.300
[1,   736] loss: 0.304
[1,   782] loss: 0.292
[1,   828] loss: 0.267
[2,    46] loss: 0.258
[2,    92] loss: 0.232
[2,   138] loss: 0.235
[2,   184] loss: 0.238
[2,   230] loss: 0.256
[2,   276] loss: 0.276
[2,   322] loss: 0.253
[2,   368] loss: 0.244
[2,   414] loss: 0.240
[2,   460] loss: 0.232
[2,   506] loss: 0.237
[2,   552] loss: 0.249
[2,   598] loss: 0.252
[2,   644] loss: 0.243
[2,   690] loss: 0.250
[2,   736] loss: 0.240
[2,   782] loss: 0.263
[2,   828] loss: 0.240
Finished Training


# Accuracy

In [21]:
correct = 0
total = 0
with torch.no_grad():
    for data in tloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the train images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the train images: 95 %


In [22]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 81 %


# Load and Save Model

In [0]:
model.load_state_dict(torch.load('/content/gdrive/My Drive/caltech10.pth'))

In [0]:
torch.save(model.state_dict(), '/content/gdrive/My Drive/caltech32.pth')

# Data Unzip and Splitting

In [0]:
#!tar -xvf '/content/gdrive/My Drive/CaltechZipped/256_ObjectCategories.tar'
#!unzip '/content/Test'

import shutil
import os
os.remove('/content/256_ObjectCategories/198.spider/RENAME2')
#shutil.rmtree('/content/256_ObjectCategories/257.clutter')
#shutil.rmtree('/content/Test/test/257.clutter')

In [12]:
import os
import os.path
import shutil
import numpy as np

myRoot = '/content/256_ObjectCategories'
dest = '/content/Test/test/'
classes = [d for d in os.listdir(myRoot)]
classes.sort()
i=0
for imgFolder in classes:
  src = os.path.join(myRoot, imgFolder)
  for image in os.listdir(src):
    # for 10% (z from Standard normal distribution curve)
    if np.random.randn(1)<-1.285:
      i=i+1
      if not os.path.exists(dest + imgFolder):
        os.mkdir(dest + imgFolder)
      shutil.move(src +'/'+ image , dest + imgFolder)
      
print(i)
      
    

3026


# Debugging Model

In [0]:
import torch.nn.functional as F
import matplotlib.pyplot as plt

model.to("cpu")
d = iter(tloader)
x,y = d.next()
#x.to(device)
#y.to(device)

out= model(x)
loss2=criterion(out,y)
print(loss2)
print(out.shape)
print(torch.argmax(out[0], dim=-1))
x=x[0,:,:,:]
x = x.numpy()
plt.imshow(np.transpose(x, (1, 2, 0)))
plt.show()

#print(torch.argmax(out, dim=1).shape)
#out2 = out.detach().numpy()
#out3 = np.argmax(out2,axis=1)
#out3 = np.squeeze(out3)

#print(out2.shape)
#print(out3.shape)
#print(out3)
#print(np.unique(out3))
#print(np.unique(y.numpy()))